# Introduction to LLM Agents with LangChain
## Solutions: Tools

**Content:**
1. [Exercise 1 (a): Explore tool parameters](#1a)
2. [Exercise 1 (b): Run tool and explore output](#1b)
3. [Exercise 2 (a): Build your own Weather tool](#2a)
4. [Exercise 2 (b): Build your own Image tool](#2b)

In [7]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('.'))
folder_b_path = os.path.join(current_dir, 'helper_functions')
sys.path.append(current_dir)

import requests
from helper_functions.keys import WEATHER_KEY, HUGGING_FACE_KEY
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from PIL import Image
import io

---

#### Exercise 1 (a): Explore tool parameters <a id='1a'></a>

**Task:**  
Use the methods ``name``, ``description``, ``args``, ``return_direct``, ``metadata`` to familiarize yourself with the parameters of the tool. What is the meaning of the different parameters?

**Background:** 
Each tool is a ``BaseTool`` class object, you can find its definition [here](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool).

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

**SOLUTION:**

In [3]:
print("Name: ", wiki_tool.name)
print("Description: ", wiki_tool.description)
print("Input argument schema: ", wiki_tool.args)
print("Return output to user? ", wiki_tool.return_direct)
print("Metadata: ", wiki_tool.metadata)

Name:  wikipedia
Description:  A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Input argument schema:  {'query': {'title': 'Query', 'description': 'query to look up on wikipedia', 'type': 'string'}}
Return output to user?  False
Metadata:  None


---

#### Exercise 1 (b): Run tool and explore output <a id='1b'></a>

**TASK:**
* Use the ``.run(tool_input)`` method to execute the tool. The ``tool_input`` is the search term that you'd like to query wikipedia with.
* [Optional] Check out the arguments of the WikipediaAPIWrapper [here](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html) and modify its parameters above. How does the output change? 

**SOLUTION:**

In [11]:
tool_input = 'pyladies'

print(wiki_tool.run(tool_input))

Page: PyLadies
Summary: PyLadies is an international mentorship group which focuses on helping more women become active participants in the Python open-source community. It is part of the Python Software Foundation. It was started in Los Angeles in 2011. The mission of the group is to create a diverse Python community through outreach, education, conferences and social gatherings. PyLadies also provides funding for women to attend open source conferences. The aim of PyLadies is increasing the participation of women in computing. PyLadies became a multi-chapter organization with the founding of the Washington, D.C., chapter in August 2011.


---

#### Exercise 2 (a): Build your own Weather tool <a id='2a'></a>
The goal is to build a tool that extracts weather information from the weather site visualcrossing.com. You typically need an API key to extract information from a website. In this example we provide you with the API key. 

**TASK:** 
- Build the tool by defining the input parameters and the descriptions. The tool function is already provided to you. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``.
- Test if the tool gives an output.

**SOLUTION:**

In [8]:
# Define the function
def extract_city_weather(city:str)->str:

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={WEATHER_KEY}&unitGroup=metric"

    response = requests.get(url)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    city: str = Field(description="City name")


# the tool description
description: str = (
        "Allows to extract the current temperature in a specific city"
    )

# fuse the function, input parameters and description into a tool. 
weather_tool = StructuredTool.from_function(
    func=extract_city_weather,
    name="weather",
    description=description,
    args_schema=WeatherInput,
    return_direct=False,
)

# test the output of the tool
print(weather_tool.run('Amsterdam'))

Current temperature in Amsterdam: 16.1°C


---

#### Exercise 2 (b): Build your own Image tool <a id='2b'></a>
The goal is to build a tool that generates an image based on a given prompt. **That means that later when you can build the Agent you can have an LLM that only outputs text, but also images!**  

To develop this, you can make use of `mobius`, text-to-image model available on HuggingFace. We provided a HuggingFace token (that you loaded in the start). 

**TASK:** 
- Build the tool by defining the input parameters and the descriptions. The tool function is already provided to you. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``.
- Test if the tool gives an output.

**SOLUTION:**

In [15]:
def text_to_image(payload:str):

    # Call the text-to-image API with the provided palaod
    API_URL = "https://api-inference.huggingface.co/models/Corcelio/mobius"
    headers = {"Authorization": f"Bearer {HUGGING_FACE_KEY}"}

    def query(payload):
        response = requests.post(API_URL, headers=headers, json=payload)
        return response.content
    
    image_bytes = query({
        "inputs": payload,
    })

    image = Image.open(io.BytesIO(image_bytes))
    
    # Resize the image
    new_size = (400, 400)  # Example new size (width, height)
    resized_image = image.resize(new_size)


    # Save the resized image to a file
    image_path = f'images/image_{payload.replace(" ", "_")}.jpg'
    resized_image.save(image_path)
    
    # Return the path to the saved image
    return f'{image_path} '


# Input parameter definition
class ImageInput(BaseModel):
    payload: str = Field(description="What should be converted into image")


# the tool description
images_tool_description: str = (
       "Generate an image based on the input text and return its path"
    )

# fuse the function, input parameters and description into a tool. 
image_tool = StructuredTool.from_function(
        func=text_to_image,
        name="create_image",
        description=images_tool_description,
        args_schema=ImageInput,
        return_direct=False,
)
image_tool.run('cat in a white box')

'images/image_cat_in_a_white_box.jpg '

---